In [3]:
import numpy as np
import math

In [4]:
# to make this problem simple, we will use for loop while calculating.

In [6]:
image = np.random.randint(0, 256, size=(1000, 1000))

In [7]:
#leveling
level = 3
image = (image/(256/level)).astype(int)

In [11]:
# normal GLCM(2D-if-counting)
# O(n^2)

def GLCM_Normal(image, level = 3, distance = 2, angle = math.pi):
    rows = image.shape[0]
    cols = image.shape[1]
    
    row = int(round(np.sin(angle))) * distance
    col = int(round(np.cos(angle))) * distance
    
    if col > 0:
        subarray_1 = image[:rows-row, :cols-col]
        subarray_2 = image[row:, col:]
    else:
        subarray_1 = image[:rows-row, -col:]
        subarray_2 = image[row:, :cols+col]
    
    GLCM_Matrix = np.zeros(shape=(level, level))
    
    print(subarray_1.shape[0])
    print(subarray_1.shape[1])
    
    for i in range(subarray_1.shape[0]):
        for j in range(subarray_1.shape[1]):
            left_value = subarray_1[i, j]
            right_value = subarray_2[i, j]
            
            GLCM_Matrix[left_value, right_value] += 1
    
    return GLCM_Matrix
            
    
        
    
    
    
    

In [10]:
%%time
GLCM_Normal(image)

1000
998
CPU times: user 888 ms, sys: 6.76 ms, total: 895 ms
Wall time: 894 ms


array([[ 112308.,  111622.,  111262.],
       [ 111351.,  110154.,  110176.],
       [ 111528.,  109873.,  109726.]])

In [17]:
# Lagrange Version
# O(n^2 * level^3)
def GLCM_Lagrange(image, level = 3, distance = 2, angle = math.pi):
    
    # same as above
    rows = image.shape[0]
    cols = image.shape[1]
    
    row = int(round(np.sin(angle))) * distance
    col = int(round(np.cos(angle))) * distance
    
    if col > 0:
        subarray_1 = image[:rows-row, :cols-col]
        subarray_2 = image[row:, col:]
    else:
        subarray_1 = image[:rows-row, -col:]
        subarray_2 = image[row:, :cols+col]
    
    GLCM_Matrix = np.zeros(shape=(level, level))
    
    for i in range(subarray_1.shape[0]):
        for j in range(subarray_1.shape[1]):
            for k in range(level):
                for l in range(level):
                    #initial of results
                    Left_result = 1
                    Right_result = 1
                    for m in range(level):
                        if(k != m):
                            Left_result *= (subarray_1[i, j]-m)/(k-m)
                        if(l != m):
                            Right_result *= (subarray_2[i, j]-m)/(l-m)
                    
                    result = Left_result*Right_result
                    GLCM_Matrix[k, l] += result
    
    return GLCM_Matrix
                
    
    
    

In [16]:
%%time
GLCM_Lagrange(image)

CPU times: user 34.7 s, sys: 96.2 ms, total: 34.8 s
Wall time: 34.9 s


array([[ 112308.,  111622.,  111262.],
       [ 111351.,  110154.,  110176.],
       [ 111528.,  109873.,  109726.]])

圖上總共有$n^2$個數對，每個數對可能有$level^2$種不同情況，所以每個數對要跑$level^2$次Lagrange。

以 $ f(0) = y_0, f(1) = y_1, ... ,f(level) = y_{level} $為例，單個Lagrange的運算是$O(level^2)$。

（分數運算花一個level，再加上分數和$ y_i $的乘積和，再花一個level)

所以最後會是$O(n^2 * level^4)$

而由於我們的y只有0跟1兩種結果，0的乘積自動為0，1也只有一項，所以本來就是$O(level)$的運算了，

用 fast-multipole也不會比較快。

總的來說，這個做法比count慢的原因有兩個：

1. 每個數對要檢查 $level^2$ 次
2. 每個Lagrange要花 $level^1$ 的時間檢查。

但如果矩陣夠大，level夠小，好像也沒關係(？)。